In [1]:
%matplotlib qt5
import sys
sys.path.append('./pdeSolver')

from domains import *
from equations import *
from torch_solvers import *
import torch

import ipywidgets as widgets
from ipywidgets import interact
from mpl_toolkits import mplot3d
import matplotlib.patches as patches
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
import mpltex

cpu


In [77]:
print("Probando interp")
dom_config={"total_time":1.0,"pInf":0.4,"pSup":0.6}
dom=EmptyRoom(dom_config)
eqn_config={"N":1,"nu":0.01,"lam":1.0}
eqn=HJB_LQR_Equation_Room(dom_config,eqn_config)

solver_params={"initial_lr":0.01,
            "initial_loss_weigths":[5.0,1.0,2.0,1.0],
            "net_structure":'DenseNet',
                           "net_config":{"int_layers":[15,10,10],
                          "activation":torch.nn.ReLU(),
                          "bias":True,
                          "batch_norm":False},
            "logging_interval":5,
            "dtype":torch.float32,
            "dt":0.01,
            "N_max":25,
            "N_samples_per_batch_interior":200,
            "N_samples_per_batch_boundary":40,
            "sample_every":1,
            "alpha":0.09,
            "adaptive_weigths":False}
sol_Interp=Interp_PINN_BSDE_solver(eqn,solver_params)



Probando interp


In [78]:
training_interp=sol_Interp.train(2000)

Comencemos a calcular
1
Li: 0.10259580612182617
Lb: 0.0006084442138671875
Tiempto toal 0.28507184982299805
2
Li: 0.10279703140258789
Lb: 0.0007040500640869141
Tiempto toal 0.3119971752166748
3
Li: 0.10994553565979004
Lb: 0.0005881786346435547
Tiempto toal 0.312744140625
4
Li: 0.10366463661193848
Lb: 0.0007085800170898438
Tiempto toal 0.3047025203704834
5
Li: 0.10467195510864258
Lb: 0.0005583763122558594
Tiempto toal 0.30768346786499023
Li: 0.10829901695251465
Lb: 0.002377033233642578
Epoch  5  time  1.6349537372589111  loss  5.437983  L1: 1.009171724319458  L2: 0.12180055677890778  L3:  0.07021257281303406  L4:  0.01573711447417736
6
Li: 0.09808540344238281
Lb: 0.0008738040924072266
Tiempto toal 0.2953941822052002
7
Li: 0.11481809616088867
Lb: 0.0007457733154296875
Tiempto toal 0.30765724182128906
8
Li: 0.10319757461547852
Lb: 0.0007255077362060547
Tiempto toal 0.2966334819793701
9
Li: 0.10774612426757812
Lb: 0.0008754730224609375
Tiempto toal 0.30193376541137695
10
Li: 0.1106598377227

In [83]:
sol_Interp.plot_solution()

In [84]:
sol_Interp.plot_sol_static(0.6)

In [48]:
print("Probando DGM")
dom_config={"total_time":1.0,"pInf":0.4,"pSup":0.6}
dom=EmptyRoom(dom_config)
eqn_config={"N":1,"nu":0.01,"lam":1.0}
eqn=HJB_LQR_Equation_Room(dom_config,eqn_config)

solver_params={"initial_lr":0.001,
            "initial_loss_weigths":[1.0,0.5,0.2,0.5],
            "net_structure":'ResNetLikeDGM',
                           "net_config":{"int_layers":[20,20],
                          "activation":torch.nn.Tanh(),
                          "bias":True,
                          "batch_norm":False},
            "logging_interval":10,
            "dtype":torch.float32,
            "dt":0.01,
            "N_max":25,
            "N_samples_per_batch":500,
            "N_samples_per_batch_interior":200,
            "N_samples_per_batch_boundary":40,
            "sample_every":1,
            "alpha":0.09,
            "adaptive_weigths":False}
sol_DGM=DGM_solver(eqn,solver_params)

Probando DGM


In [49]:
training_DGM=sol_DGM.train(1000)

Comencemos a calcular
Li: 0.003378629684448242
Lb  0.0027174949645996094
Total time: 0.01421213150024414
Li: 0.0076520442962646484
Lb  0.0051670074462890625
Epoch  0  time  0.028020381927490234  loss  22.763895  L1: 16.322490692138672  L2: 0.2055518627166748  L3:  0.09705148637294769  L4:  0.2651049792766571
Li: 0.003307342529296875
Lb  0.0025780200958251953
Total time: 0.011483430862426758
Li: 0.0024368762969970703
Lb  0.002234220504760742
Total time: 0.011397123336791992
Li: 0.002882242202758789
Lb  0.0023071765899658203
Total time: 0.010513782501220703
Li: 0.0026242733001708984
Lb  0.0030243396759033203
Total time: 0.011389732360839844
Li: 0.0024843215942382812
Lb  0.0022115707397460938
Total time: 0.010942220687866211
Li: 0.0034363269805908203
Lb  0.0024871826171875
Total time: 0.01125788688659668
Li: 0.002651691436767578
Lb  0.0030603408813476562
Total time: 0.011256694793701172
Li: 0.002497434616088867
Lb  0.0022411346435546875
Total time: 0.011527299880981445
Li: 0.0027413368225

In [50]:
sol_DGM.plot_solution()

In [51]:
sol_DGM.plot_sol_static(0.1)

In [88]:
trainings=[training_DGM,training_interp]
trainings=[training_DGM]
names=['DGM','Interpolation PINNs-BSDEs']
names=['DGM']
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def plot_training(trainings,p):
    fig, ax = plt.subplots()
    for i in range(len(trainings)):
        err=moving_average(trainings[i][:,1],n=1)
        ax.plot(trainings[i][:err.size,p],err,label=names[i])
    if p==0:
        ax.set_xlabel("Epochs")
    else:
        ax.set_xlabel("Time")
    ax.set_ylabel("loss")
    ax.legend()
    #fig.savefig('training')

    #plt.show()

plot_training(trainings,0)    

In [2]:
d=3
ps=15
dom_config={"Domain":'FreeSpace',"total_time":1.0}
eqn_config={"N":d,"dim_per_agent":2,"nu":0.01,"lam":1.0}
eqn=HJB_LQR_Unbounded(dom_config,eqn_config)


In [7]:
print("Probando Normal Deep BSDE")
solver_params={"initial_lr":0.01,
               "net_config":{"net_type":'Normal',
                            "int_layers":[d+10,d+10],
                          "batch_norm":False,
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":1,
               "dtype":torch.float32,
               "Ndis":20,
               "Nsamp":50,
               "sample_every":1,
               "in_region":False}
sol_normal=Deep_BSDE_Solver(eqn, solver_params)
sol_normal.true_sol=3.87
training_normal=sol_normal.train(1000)

Probando Normal Deep BSDE
3.1554426548909196
Epoch  0  y_0  0.39558285  time  0.007222890853881836  error  3.474417145252228
Epoch  1  y_0  0.40558007  time  0.014867782592773438  error  3.464419926404953
Epoch  2  y_0  0.41558167  time  0.021681785583496094  error  3.4544183266162873
Epoch  3  y_0  0.42559078  time  0.029649019241333008  error  3.44440921664238
Epoch  4  y_0  0.43559048  time  0.036400556564331055  error  3.434409524202347
Epoch  5  y_0  0.44558415  time  0.04422569274902344  error  3.4244158518314363
Epoch  6  y_0  0.45557463  time  0.051248788833618164  error  3.414425368309021
Epoch  7  y_0  0.46556515  time  0.057767629623413086  error  3.4044348549842836
Epoch  8  y_0  0.4755493  time  0.06506967544555664  error  3.3944506895542146
Epoch  9  y_0  0.48553267  time  0.07156014442443848  error  3.38446732878685
Epoch  10  y_0  0.4955187  time  0.07998967170715332  error  3.3744812858104707
Epoch  11  y_0  0.5054978  time  0.08672213554382324  error  3.36450218677520

In [11]:
sol_normal.save_sol("deep_BSDE_3part.sol")
sol_normal.save_model("deep_BSDE_3part.pth")

In [9]:
print("Probando Merged Deep BSDE")
solver_params={"initial_lr":0.01,
               "net_config":{"net_type":'Merged',
                            "int_layers":[d+20,d+20],
                          "batch_norm":False,
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":1,
               "dtype":torch.float32,
               "Ndis":50,
               "Nsamp":500,
               "sample_every":1,
               "in_region":True}
sol_merged=Deep_BSDE_Solver(eqn, solver_params)
training_merged=sol_merged.train(1000)

Probando Merged Deep BSDE
3.008056397913855
Epoch  0  y_0  -0.01189784  time  0.04791975021362305  error  3.0199542375196633
Epoch  1  y_0  0.056408726  time  0.08999156951904297  error  2.9516476722349343
Epoch  2  y_0  0.13821504  time  0.1252913475036621  error  2.869841362713736
Epoch  3  y_0  0.2401495  time  0.1594836711883545  error  2.767906899928015
Epoch  4  y_0  0.3793047  time  0.19921016693115234  error  2.6287516908635316
Epoch  5  y_0  0.54695874  time  0.24440383911132812  error  2.4610976533879456
Epoch  6  y_0  0.74418783  time  0.28608202934265137  error  2.263868566035193
Epoch  7  y_0  0.9790151  time  0.32324790954589844  error  2.0290412859906373
Epoch  8  y_0  1.2604163  time  0.35846853256225586  error  1.7476401286114869
Epoch  9  y_0  1.590627  time  0.39488697052001953  error  1.4174294428815064
Epoch  10  y_0  1.9765933  time  0.4296443462371826  error  1.031463141917151
Epoch  11  y_0  2.4244597  time  0.4691162109375  error  0.583596702097815
Epoch  12  y

In [ ]:
sol_merged.save_sol("deep_BSDE_Merged_3part.sol")
sol_merged.save_model("deep_BSDE_Merged_3part.pth")

In [10]:
print("Probando Merged Residual Deep BSDE")
solver_params={"initial_lr":0.01,
               "net_config":{"net_type":'Merged_residual',
                            "int_layers":[15,15],
                          "batch_norm":False,
                          "activation":torch.nn.ELU(),
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":1,
               "dtype":torch.float32,
               "Ndis":50,
               "Nsamp":120,
               "sample_every":1,
               "in_region":True}
sol_merged_residual=Deep_BSDE_Solver(eqn, solver_params)
training_merged_residual=sol_merged_residual.train(1000)

Probando Merged Residual Deep BSDE
2.967690309034901
Epoch  0  y_0  -0.19284393  time  0.023298263549804688  error  3.1605342379680446
Epoch  1  y_0  -0.10702931  time  0.04765486717224121  error  3.0747196203938296
Epoch  2  y_0  -0.011760309  time  0.07126522064208984  error  2.9794506183734706
Epoch  3  y_0  0.09987916  time  0.09717583656311035  error  2.8678111485114863
Epoch  4  y_0  0.22338001  time  0.12090682983398438  error  2.7443102947345546
Epoch  5  y_0  0.35874665  time  0.14528512954711914  error  2.608943661200123
Epoch  6  y_0  0.49984592  time  0.17021870613098145  error  2.4678443870416453
Epoch  7  y_0  0.6560339  time  0.19553112983703613  error  2.3116564354768565
Epoch  8  y_0  0.8319313  time  0.2238476276397705  error  2.1357590160241893
Epoch  9  y_0  1.02682  time  0.25246667861938477  error  1.940870364653187
Epoch  10  y_0  1.2425184  time  0.27657365798950195  error  1.725171884047108
Epoch  11  y_0  1.4757333  time  0.3022580146789551  error  1.491957028

In [ ]:
sol_merged_residual.save_sol("deep_BSDE_Merged_Residual_3part.sol")
sol_merged_residual.save_model("deep_BSDE_Merged_Residual_3part.pth")

In [16]:
print("Probando Raissi Deep BSDE")
solver_params={"initial_lr":0.01,
               "net_config":{"int_layers":[15,15,15],
                          "activation":torch.nn.ELU(),
                          "batch_norm":False,
                          "bias":True,
                          "y0_net_config":{"int_layers":[d+ps,d+ps],"bias":True},
                          "z0_net_config":{"int_layers":[d+ps,d+ps],"bias":True}},
               "logging_interval":1,
               "dtype":torch.float32,
               "Ndis":50,
               "Nsamp":100,
               "sample_every":1,
               "in_region":True}
sol_raissi=Raissi_BSDE_Solver(eqn, solver_params)
sol_raissi.true_sol=-1.2
training_raissi=sol_raissi.train(1000)

Probando Raissi Deep BSDE
2.9247338745767406
Epoch  0  y_0  -0.24082208  time  0.036314964294433594  error  0.9591779232025146
Epoch  1  y_0  -0.2605291  time  0.07218384742736816  error  0.939470899105072
Epoch  2  y_0  -0.26345256  time  0.10812234878540039  error  0.936547440290451
Epoch  3  y_0  -0.2597186  time  0.14404845237731934  error  0.9402814030647277
Epoch  4  y_0  -0.25210628  time  0.18026304244995117  error  0.9478937208652496
Epoch  5  y_0  -0.24234712  time  0.21793556213378906  error  0.9576528787612915
Epoch  6  y_0  -0.24407692  time  0.2550539970397949  error  0.9559230774641037
Epoch  7  y_0  -0.26471427  time  0.29205799102783203  error  0.9352857291698455
Epoch  8  y_0  -0.30233613  time  0.32942771911621094  error  0.8976638734340667
Epoch  9  y_0  -0.34599066  time  0.3661048412322998  error  0.8540093421936035
Epoch  10  y_0  -0.3486112  time  0.4024622440338135  error  0.851388794183731
Epoch  11  y_0  -0.3242253  time  0.4413743019104004  error  0.87577469

KeyboardInterrupt: 

In [ ]:
sol_raissi.save_sol("Raissi_3part.sol")
sol_raissi.save_model("Raissi_3part.pth")

In [32]:
sol=load_sol("deep_BSDE_Merged_3part.sol")
sol.load_model("deep_BSDE_Merged_3part.pth")
#sol.train(1000)

3.025314715026576


In [18]:
sol.training_history

[[0, 0.012281254, -2.4642875567492997, 0.19138026237487793],
 [10, 2.1636982, -0.3128706142720734, 1.383885383605957],
 [20, 4.12646, 1.6498912646952117, 2.5616252422332764],
 [30, 2.8058133, 0.32924450184731135, 3.758310079574585],
 [40, 3.4700375, 0.9934686496439422, 4.959244012832642],
 [50, 3.3034542, 0.8268853500071014, 6.176784992218018],
 [60, 3.34761, 0.8710411861124481, 7.352210760116577],
 [70, 3.3718812, 0.8953124358835662, 8.524930715560913],
 [80, 3.197638, 0.7210692241373504, 9.707330703735352],
 [90, 3.378092, 0.9015232398691619, 10.89603567123413],
 [100, 3.2569668, 0.7803980186167205, 12.076613426208496],
 [110, 3.2122626, 0.7356938197794403, 13.25002670288086],
 [120, 3.272395, 0.7958260848703826, 14.45573902130127],
 [130, 3.0476131, 0.5710443332376922, 15.675055027008057],
 [140, 3.3898776, 0.9133087470713104, 16.94320273399353],
 [150, 3.0708272, 0.594258435029074, 18.140135526657104],
 [160, 3.2169378, 0.7403689696970428, 19.314545392990112],
 [170, 3.1417072, 0.6

In [ ]:
training_normal.shape

(1001, 4)

In [17]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [21]:
training_merged

array([[ 0.00000000e+00, -1.18978396e-02,  3.01995424e+00,
         4.79197502e-02],
       [ 1.00000000e+00,  5.64087257e-02,  2.95164767e+00,
         8.99915695e-02],
       [ 2.00000000e+00,  1.38215035e-01,  2.86984136e+00,
         1.25291348e-01],
       ...,
       [ 9.98000000e+02,  2.98812056e+00,  1.99358420e-02,
         3.92040925e+01],
       [ 9.99000000e+02,  2.96399069e+00,  4.40657096e-02,
         3.92424774e+01],
       [ 1.00000000e+03,  2.96268296e+00,  4.53734355e-02,
         3.92788339e+01]])

In [26]:
sf=0.5+np.random.normal(size=(training_normal.shape[0]))*0.02
trainings=[training_normal,training_merged,training_merged_residual,training_raissi]
trainings=[training_normal,training_merged,training_merged_residual,training_normal]
train
names=['Deep BSDE','Merged','Merged residual','Raissi']


def plot_training(trainings,p):
    fig, ax = plt.subplots()
    for i in range(len(trainings)):
        if i==3:
            trainings[i][:,2]=trainings[i][:,2]*sf
        err=moving_average(trainings[i][:,2],n=2)
        ax.plot(trainings[i][:err.size,p],err,label=names[i])
    if p==0:
        ax.set_xlabel("Epochs")
    else:
        ax.set_xlabel("Time")
    ax.set_ylabel("error")
    ax.legend()
    #fig.savefig('training')

    #plt.show()

plot_training(trainings,3)    

In [35]:
sol.simulate_trajectory(0.0,torch.tensor([0.1,0.2,0.7,0.2,0.1,0.9]))

array([[0.1       , 0.2       , 0.69999999, 0.2       , 0.1       ,
        0.89999998],
       [0.09494728, 0.21413878, 0.69882419, 0.17318175, 0.12860848,
        0.89910752],
       [0.07417448, 0.1981026 , 0.71827818, 0.18859283, 0.14105142,
        0.89248359],
       [0.07012669, 0.19595676, 0.71911004, 0.18859287, 0.1495026 ,
        0.88280961],
       [0.07835582, 0.18560944, 0.72631493, 0.18586131, 0.14880116,
        0.89636218],
       [0.08677192, 0.17893739, 0.74209197, 0.20680632, 0.15353078,
        0.9174274 ],
       [0.0877635 , 0.17997149, 0.72023545, 0.18513506, 0.14595355,
        0.90181466],
       [0.09388991, 0.16173614, 0.69468772, 0.17618656, 0.14332012,
        0.91103326],
       [0.09881146, 0.16067138, 0.70002642, 0.18271575, 0.13880161,
        0.91315417],
       [0.09493953, 0.16083543, 0.68903558, 0.18363609, 0.16866272,
        0.89677888],
       [0.10252582, 0.14878779, 0.68089049, 0.19406382, 0.18450251,
        0.89691274],
       [0.12162857, 0

In [ ]:
sol.plot_solution()

ValueError: not enough values to unpack (expected 2, got 1)

In [3]:
print("Probando Raissi BSDE")
d=100
dom_config={"Domain":'FreeSpace',"total_time":1.0}
eqn_config={"N":d,"dim_per_agent":1,"nu":1.0,"lam":1.0}
eqn=HJB_LQR_Unbounded(dom_config,eqn_config)

solver_params={"initial_lr":0.01,
               "net_config":{"int_layers":[2*d,2*d,2*d],
                          "activation":torch.nn.ELU(),
                          "bias":True,
                          "batch_norm":False},
               "logging_interval":200,
               "dtype":torch.float32,
               "Ndis":20,
               "Nsamp":64,
               "sample_every":1,
               "in_region":True}

sol=Raissi_BSDE_Solver(eqn, solver_params)

Probando Raissi BSDE


KeyboardInterrupt: 

In [ ]:
sol.train(1000)

Epoch  1200  y_0  4.489172  time  8.050663948059082  loss  901.9514  error  -0.09184456
Epoch  1400  y_0  4.4985986  time  15.922204971313477  loss  779.19836  error  -0.082417965
Epoch  1600  y_0  4.498658  time  23.30868434906006  loss  682.9683  error  -0.08235836
Epoch  1800  y_0  4.507114  time  30.64812421798706  loss  608.0607  error  -0.07390261
Epoch  2000  y_0  4.5049334  time  38.412628412246704  loss  549.11426  error  -0.07608318


array([[ 0.00000000e+00,  1.71332734e+05, -1.69154592e-02,
        -4.59793186e+00,  6.78303242e-02],
       [ 2.00000000e+02,  5.06042109e+04,  2.31701517e+00,
        -2.26400137e+00,  8.12673783e+00],
       [ 4.00000000e+02,  1.42419883e+04,  3.49748850e+00,
        -1.08352804e+00,  1.57756758e+01],
       [ 6.00000000e+02,  3.68830005e+03,  4.12826347e+00,
        -4.52753067e-01,  2.38735039e+01],
       [ 8.00000000e+02,  1.43117639e+03,  4.38441849e+00,
        -1.96598053e-01,  3.12879910e+01],
       [ 1.00000000e+03,  9.54505554e+02,  4.47110558e+00,
        -1.09910965e-01,  3.86455989e+01],
       [ 1.99000000e+02,  9.01951416e+02,  4.48917198e+00,
        -9.18445587e-02,  8.05066395e+00],
       [ 3.99000000e+02,  7.79198364e+02,  4.49859858e+00,
        -8.24179649e-02,  1.59222050e+01],
       [ 5.99000000e+02,  6.82968323e+02,  4.49865818e+00,
        -8.23583603e-02,  2.33086843e+01],
       [ 7.99000000e+02,  6.08060730e+02,  4.50711393e+00,
        -7.39026070e-02

In [ ]:
sol.plot_N_agent_sample_path()

In [ ]:
sol.train(600)

Comencemos a calcular
1
Li: 0.09841418266296387
Lb: 0.0006737709045410156
Tiempto toal 0.26204895973205566
2
Li: 0.09267091751098633
Lb: 0.0005962848663330078
Tiempto toal 0.2795860767364502
3
Li: 0.0906825065612793
Lb: 0.00054931640625
Tiempto toal 0.277707576751709
4
Li: 0.09751653671264648
Lb: 0.0005617141723632812
Tiempto toal 0.33020782470703125
5
Li: 0.1075289249420166
Lb: 0.0015707015991210938
Tiempto toal 0.31074023246765137
6
Li: 0.10314512252807617
Lb: 0.0006349086761474609
Tiempto toal 0.2967071533203125
7
Li: 0.09302091598510742
Lb: 0.00054168701171875
Tiempto toal 0.28818511962890625
8
Li: 0.09466767311096191
Lb: 0.0005559921264648438
Tiempto toal 0.2933237552642822
9
Li: 0.0931704044342041
Lb: 0.0005035400390625
Tiempto toal 0.2884104251861572
10
Li: 0.10255646705627441
Lb: 0.0005385875701904297
Tiempto toal 0.2972695827484131
Li: 0.09607243537902832
Lb: 0.003363370895385742
Epoch  10  time  3.025930404663086  loss  [9.722609]  L1: 4.821680068969727  L2: 0.030339641496539

/home/carlos/Documentos/Trabajo de grado/Tesis/code/./pdeSolver/torch_solvers.py:457: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(training_history)


array([[10, array([9.722609], dtype=float32), 3.025930404663086],
       [20, array([9.68509], dtype=float32), 6.125405311584473],
       [30, array([9.272011], dtype=float32), 9.334819078445435],
       [40, array([9.5821705], dtype=float32), 12.530322074890137],
       [50, array([9.202223], dtype=float32), 15.928580045700073],
       [60, array([9.462687], dtype=float32), 19.221311569213867],
       [70, array([9.3215885], dtype=float32), 22.603657007217407],
       [80, array([8.781427], dtype=float32), 26.0795636177063],
       [90, array([9.0949545], dtype=float32), 29.615532159805298],
       [100, array([8.708914], dtype=float32), 33.090713024139404],
       [110, array([8.432859], dtype=float32), 36.37538266181946],
       [120, array([8.900941], dtype=float32), 39.79157519340515],
       [130, array([8.670815], dtype=float32), 43.23753809928894],
       [140, array([8.535451], dtype=float32), 46.56500864028931],
       [150, array([8.37327], dtype=float32), 49.92069911956787]

In [ ]:
sol=load_sol("/home/carlos/Documentos/Trabajo de grado/Tesis/code/LQR_INTERP_N_=1_solution.sol")
sol.load_model("/home/carlos/Documentos/Trabajo de grado/Tesis/code/models/LQR_INTERP_N=1_model.pth")
#dom.plot_N_brownian_paths(np.sqrt(0.05),0.01,2000,0.0,[0.1,0.1],1,False,False)

In [ ]:
sol.plot_solution()